In [1]:
# Imports you may need
import seaborn as sns
from IPython.display import display, HTML
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import yfinance as yf
import json
import bz2
from dataloader import *
from finance import *
%load_ext autoreload
%autoreload 2

In [ ]:
# Cellule pour importer les questions qu'on aura fait individuellement, et sera supprimée à la fin.

from section1 import *
from finance import applestock
from quotebankexploration import *

In [ ]:
quotes = load_quotes(r"data/quotes-2018.json.bz2", limit = 1000, columns = ["quoteID", "speaker","date"])

## Quotebank Data Exploration

In [ ]:
quotebank_exploration(quotes)

The dataset comprises the following columns : 
 * `quoteID`: Primary key of the quotation (format: "YYYY-MM-DD-{increasing int:06d}").
 * `quotation`: Text of the longest encountered original form of the quotation.
 * `speaker`: Selected most likely speaker. This matches the first speaker entry in `probas`. If none of the speakers is selected, the speaker is defined as "None".
 * `qids`: Wikidata IDs of all aliases that match the selected speaker. If no Wikidata IDs is found, the value is '[]'.
 * `date`: Earliest occurrence date of any version of the quotation.
 * `numOccurrences`: Number of time this quotation occurs in the articles.
 * `probas`: Array representing the probabilities of each speaker having uttered the quotation.
 * `urls`: List of links to the original articles containing the quotation. 
 * `phase`: Corresponding phase of the data in which the quotation first occurred (A-E).

## Quotebank Data Pre-processing

In [ ]:
obama_quotes = filter_quotes(r"data/quotes-2008.json.bz2", speakers = ['obama'], keywords=["yes we can"], chunknum = 1000, save="obama_quotes")

# Stock
This is an example

In [ ]:
applestock()

In [5]:
keywords_1_word = ["iphone", "ipad", "imac", "ipod", "macbook", "mac", "airpods", \
        "lightening", "magsafe", "aapl", "iwatch", "itunes", "homepod", "macos", "ios", "ipados", \
        "watchos", "tvos", "wwdc", "siri", "facetime", "appstore", "icloud", "iphones"]

keywords_2_words = ["apple watch", "steve jobs", "tim cook", "face id", \
        "pro display xdr", "katherine adams", "eddy cue", "craig federighi"]

keywords_capital = ["Apple"]

black_list = ["Big Apple"]

keywords = {"One word": keywords_1_word, "Two words": keywords_2_words, "Capital words": keywords_capital, "Black list": black_list}

speakers = ["steve jobs", "tim cook", "katherine adams", "eddy cue", "craig federighi", "john giannandrea", "greg joswiak", \
    "sabih khan", "luca maestri", "deirdre o'brien", "johny srouji", "john ternus", "jeff williams", "lisa jakson", \
    "stella low", "isabel ge mahe", "tor myhren", "adrian perica", "phil schiller", "arthur levinson", "james bell", \
    "albert gore", "andrea jung", "monica lozano", "ronald sugar", "susan wagner"]

In [10]:

years_gas = range(2008, 2014)
years_raph = range(2014, 2021)

for year in years_raph:
    filter_quotes(f"data/quotes-{str(year)}.json.bz2", speakers = speakers, keywords = keywords, save = f"filtered_quotes_{str(year)}")

0it [00:00, ?it/s]

INFO: 29514 citations have been kept.


0it [00:00, ?it/s]

INFO: 31719 citations have been kept.


0it [00:00, ?it/s]

INFO: 19807 citations have been kept.


0it [00:00, ?it/s]

INFO: 29489 citations have been kept.


0it [00:00, ?it/s]

KeyboardInterrupt: 